In [ ]:
import math

import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import zscore
hv.extension('bokeh')


# SciKit-Learn packages
from sklearn.metrics import hamming_loss, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

# SciKit-multi-learn
from skmultilearn.problem_transform import BinaryRelevance

# Stop words for NLP
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Custom functions used in Q2
from Q2Funcs import find_uniques, one_hot, column_score

%matplotlib inline

# Movie Revenue Predictions
In dit verslag worden de volgende drie onderzoeksvragen uitgewerkt:

1. In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
2. In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort?
3. In hoeverre is het mogelijk om de budget-winst verhouding te voorspellen?

## Het Data Science proces
Dit verslag wordt met behulp van het Data Science proces uitgewerkt, deze gaat als volgt:
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
7. Communication

Het verslag begint met een algemene Data Collection en Data Processing. De rest van de stappen worden per vraag uitgewerkt.

## 1. Data Collection
De Data Collection is deels al voor ons gedaan. De dataset `movie.csv` is ons aangeleverd. Echter word voor de opdracht 
gevraagd om deze te combineren met een dataset van derden. Om erachter te komen welke dataset geschikt is om te 
combineren met `movies.csv`zullen wij deze dataset eerst moeten processen, cleanen en exploren.  

Om te zien of de dataset `movie.csv` goed is ingeladen, worden de eerste vijf rijen getoond:

In [ ]:
df_movies = pd.read_csv('data/movie.csv')
df_movies.head()

### Externe dataset
De externe dataset is van David Batista. Het bestaat uit een plot beschrijving en een aantal genres. Het wordt gebruikt om het model in Q2 verder te testen. De dataset is te downloaden op [Document Classification](http://www.davidsbatista.net/blog/2017/04/01/document_classification/).

In [ ]:
df_genres = pd.read_csv('data/imdb/movies_genres.csv', delimiter='\t')
df_genres.head()

## 2. Data Processing

Ook deze stap is grotendeels voor ons gedaan. De data is goed opgeslagen in een `.csv`-bestand en kan direct worden 
ingelezen in een _Pandas_ DataFrame.

Verder rest ons nog de volgende vier stappen:
1. Het verwijderen van ongewenste kolommen
2. Het aanpassen van onduidelijke kolomnamen
3. Het aanpassen van de volgorde van de kolommen
4. Het aanpassen van enkele datatypen

In [ ]:
# Ongewenste kolommen verwijderen
df_movies.drop(["movie_imdb_link", "aspect_ratio"], axis=1, inplace=True)

#Onduidelijke kolomnamen aanpassen
df_movies.rename(columns={'color': 'Colour',
                          'director_name': 'Director',
                          'num_critic_for_reviews': 'Number of critics',
                          'duration': 'Duration',
                          'director_facebook_likes': 'Director FB likes',
                          'actor_3_facebook_likes': 'Actor 3 FB likes',
                          'actor_2_name': 'Actor 2 name',
                          'actor_1_facebook_likes': 'Actor 1 FB likes',
                          'gross': 'Gross',
                          'genres': 'Genres',
                          'actor_1_name': 'Actor 1 name',
                          'movie_title': 'Movie title',
                          'num_voted_users': 'Number of voted users',
                          'cast_total_facebook_likes': 'Total Cast FB likes',
                          'actor_3_name': 'Actor 3 name',
                          'facenumber_in_poster': 'Number of faces on poster',
                          'plot_keywords': 'Plot Keywords',
                          'num_user_for_reviews': 'Number of user reviews',
                          'language': 'Language',
                          'country': 'Country',
                          'content_rating': 'Age rating',
                          'budget': 'Budget',
                          'title_year': 'Release year',
                          'actor_2_facebook_likes': 'Actor 2 FB likes',
                          'imdb_score': 'IMDB Score',
                          'movie_facebook_likes': 'Movie FB likes'}, inplace=True)

# Volgorde kolommen aanpassen
df_movies = df_movies[['Movie title',
                       'Release year',
                       'Director',
                       'Director FB likes',
                       'Gross',
                       'Budget',
                       'Duration',
                       'Language',
                       'Country',
                       'Colour',
                       'Genres',
                       'IMDB Score',
                       'Number of voted users',
                       'Number of critics',
                       'Number of user reviews',
                       'Age rating',
                       'Total Cast FB likes',
                       'Movie FB likes',
                       'Actor 1 name',
                       'Actor 2 name',
                       'Actor 3 name',
                       'Actor 1 FB likes',
                       'Actor 2 FB likes',
                       'Actor 3 FB likes',
                       'Plot Keywords',
                       'Number of faces on poster',
                       ]]

# Datatypen aanpassen
# 1. Floats omzetten naar integers
#  De dataset bevat geen kolommen die dienen te worden bewaard als float, behalve `IMDB Score`
df_movies_IMDB_Score = df_movies["IMDB Score"]  # Tijdelijke kopie van de kolom `IMDB Score`
df_movies = df_movies.drop('IMDB Score', axis=1).fillna(0).astype(int, errors='ignore') # Waarden omzetten naar integers
df_movies.insert(11, "IMDB Score", df_movies_IMDB_Score)  # `IMDB Score` weer toevoegen aan originele DataFrame
del df_movies_IMDB_Score

# 2. De kolom `Release year` omzettten van integers naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y', errors='coerce')

Het DataFrame ziet er nu als volgt uit:

In [ ]:
df_movies.head()

## 3. Data Cleaning

1. Het verwijderen van NaN-types
2. Het verwijderen van dubbele filmtitles
3. Het verwijderen van negatieve waardes

In [ ]:
# NaN-types verwijderen
df_movies.dropna(inplace=True)

# Dubbele titels verwijderen
df_movies.sort_values("Release year", inplace=True)  # Sorteren op uitgavejaar
df_movies.drop_duplicates(subset="Movie title", keep="last", inplace=True)  # Alleen meest recente versie blijft bewaard

# Negatieve waardes verwijderen
num = df_movies._get_numeric_data()
num[num < 0] = 0

Na stap 3. Data Cleaning ziet het DataFrame er als volgt uit:

In [ ]:
df_movies.head()

 # Individuele onderzoeksvragen


## Q1. Omzet voorspellen op basis van de populariteit
De onderzoeksvraag gaat als volgt.

> In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
Bij deze deelvraag is besloten om de volgorde van de data science proces niet volledig aan te houden. Dat past beter bij deze deelvraag.

### Q1. Data collection

Voor deze onderzoeksvragen wordt er gebruik gemaakt van de volgende features uit `df_movies`:
* Director FB likes
* Actor 1 FB likes
* Actor 2 FB likes
* Actor 3 FB likes
* Total Cast FB likes
* Movie FB likes
* Gross
* IMDB Score

In [ ]:
# Alleen facebook kolommen of IMDB kolommen nodig
features = ['Director FB likes', 'Actor 1 FB likes', 
                'Actor 2 FB likes', 'Gross', 'Total Cast FB likes',
                'Actor 3 FB likes', 'IMDB Score', 'Movie FB likes']
df_Q1 = df_movies.copy()[features]

### Q1. Data Cleaning

Algemene Data Cleaning werd in `3. Data Cleaning` gedaan. In `Q1. Data Cleaning` wordt de data nog verder opgeschoond om te voldoen aan de eisen van deze deelvraag. 

Alle films zonder omzet verwijderd worden, deze zijn namelijk niet bruikbaar voor de voorspelling.

In [ ]:
df_Q1 = df_Q1[df_Q1.Gross != 0]

### Q1. Data Exploration & Analysis

In [ ]:
df_Q1.describe()

Er zitten grote verschillen tussen de waardes van elke kolom. Aangezien machine learning algoritmes algebraïsch zijn heeft een hoog getal meer invloed dan een laag getal. Om dit te voorkomen moeten de waardes genormaliseerd worden. Dit gebeurt in `Q1. Data processing`.

De volgende stap is (lineaire) correlaties vinden.

In [ ]:
plt.matshow(df_Q1.corr())
plt.show()
df_Q1.corr()

Wat hieruit zichtbaar wordt is:
* `Actor 1 FB likes` heeft veel invloed op `Total Cast FB likes`
* `Actor 2 FB likes` heeft aanzienlijk minder invloed dan `Actor 1 FB likes` op de `Total Cast FB likes`
* `Movie FB likes` heeft een zwakke invloed op `Gross`.


### Q1. Data processing

Vervolgens is de dataset opgesplitst naar een input `X` en output `y` waar ze vervolgens gesplitst worden naar een train, test en validatie set.

In [ ]:
# Split the data to X and y
X = df_Q1.drop(columns=['Gross'])
y = df_Q1['Gross']
assert len(X) == len(y) # make sure that the length of x is the same as the length of y



In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)




### Q1. Model building

Aangezien er sprake is van correlatie tussen de input en output is, is een regressie mogelijk.



In [ ]:
LR = LogisticRegression(random_state=1, solver="lbfgs", multi_class="auto", max_iter=200)

LR.fit(X_train, y_train)



Bij het fitten van dit model wordt een `Convergence warning` gegeven. Aangezien het aantal iteraties al erg hoog is, houdt dat in dat een logistic regression model niet past bij deze data.


De volgende stap is de waarde van de voorspelling te bepalen.

In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(abs(mse))
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(abs(mse)))
print("Score: {:.4f}".format(score))



Uit deze scores blijkt opnieuw dat dit model niet werkt bij deze data. De voorspelling is uitzonderlijk slecht.


#### Linear Regression

De volgende stap is om een ander regressie model te proberen. Er is hier gekozen voor een simpele lineaire regressie.

In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)


In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))



Deze scores zijn al aanzienlijk beter dan de scores van het logistic model. De scores zijn echter nog steeds niet écht goed.

### Q1. Data processing

Nadat er een base score is van dit model kan je normalisatie toevoegen en kijken wat de invloed hier op is.

In [ ]:
x_scaled = stats.zscore(X)
y_scaled = stats.zscore(y)

Nadat we de X data gescaled hebben door middel van een zscores moeten we dit weer op splitsen naar train, test en validatie sets.

In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)


In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)

In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))



Wat we hier bevinden:
* De score is zo goed als gelijk gebleven op vergelijking van de vorige voorspelling met een lineaire regressie
* Mean squared error is in vergelijking stevig gedaalt

In [ ]:
x = np.linspace(0, 100, len(y_test))
plt.plot(x,y_test, LR.predict(X_test)[:100])


#### Polynomial regression

Het laatste model dat getest wordt is polynomiale regressie.


In [ ]:
polynomial_features= PolynomialFeatures(degree=2)
x_poly = polynomial_features.fit_transform(x_scaled)

In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(x_poly, y_scaled, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)



In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))




Dit levert ons een slechter model op. Zo is de MSE omhoog gegaan en de Score naar beneden

### Q1. Communication
#### Q1. Definitieve pipeline
Onze definitieve model is een lineaire regressie model waarbij de X en y data gescaled is door middel van z-scores


In [ ]:
y = df_Q1['Gross']
y_scaled = stats.zscore(y)

In [ ]:
# Create Train test and validation Split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=1)
# Doing it twice because you need a validation set as well
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
LR = LinearRegression()

LR.fit(X_train, y_train)

#### Test score

In [ ]:
mse = mean_squared_error(y_test, LR.predict(X_test))
rms = math.sqrt(mse)
score = LR.score(X_test, y_test)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))



#### Validation score

In [ ]:
mse = mean_squared_error(y_val, LR.predict(X_val))
rms = math.sqrt(mse)
score = LR.score(X_val, y_val)

print("RMS: {}".format(rms))
print("MSE: {}".format(mse))
print("Score: {}".format(score))
print("Std: {}".format( df_Q1.Gross.std()))

In [ ]:
x = np.linspace(0, len(y_val), len(y_val))
plt.plot(x,y_val, LR.predict(X_val))

#### Q1. Conclusie

De onderzoeksvraag die beantwoord wordt met behulp van dit model ging als volgt:

>In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?


Het is niet gelukt om deze voorspellingen te maken door middel van een regressie model. Het beste resultaat had een Accuracy van 19%, dit is lang niet accuraat genoeg.



## Q2. Genres voorspellen gebaseerd op Plot Keywords
De onderzoeksvraag gaat als volgt: 
> In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort? 


### Q2. Data collection
Bij `Q2` wordt er gebruik gemaakt van de kolommen `Genres` en `Plot Keywords` uit `df_movies`. Om ervoor te zorgen dat bij andere onderzoeksvragen de DataFrame niet onnodig wordt aangepast, wordt de dataframe voor de zekerheid als een kopie aangeroepen. 

Ook wordt het model aan het einde van het hoofdstuk getest in een andere scenario. Er wordt getest vanuit de plot beschrijving (I.E. een alinea lange beschrijving van het plot) in plaats van plot keywords. Deze data wordt tegelijkertijd met `DF_Q2` verwerkt tot een X en een Y.

Deze extra data komt van [Document Classification](http://www.davidsbatista.net/blog/2017/04/01/document_classification/).

In [ ]:
df_Q2 = df_movies.loc[:, ("Genres", "Plot Keywords")].copy()
df_Q2.head()

### Q2. Data processing
Aangezien vrijwel alle machine learning algoritmen alleen algebraïsche datatypes accepteren, moeten zowel `Plot Keywords` als `Genres` ingrijpend veranderd worden. Beide zijn namelijk categoriale datatypes.


#### Q2. Cleaning voor processing


Na een korte inspectie van de data, kom je een aantal waardes van 0 en “0” tegen (`int` en `string`). Dit zijn missende waarden. 

Hoewel dit gewoonlijks in stap 3 van het Data Science proces gebeurt, is het in deze scenario beter om dit van tevoren te doen. De 0 en "0" zijn na deze stap namelijk moeilijker terug te vinden.



In [ ]:
# All '0' are missing values, remove
df_Q2.replace("0", np.NaN, inplace=True)
df_Q2.replace(0, np.NaN, inplace=True)
df_Q2.dropna(inplace=True)

#### Q2. Terug naar processing
Om in de gebruikte modellen gebruik te maken van `Genres` en `Plot Keywords` moeten deze gecodeerd worden. 

Om dit te doen moeten de kolommen eerst gesplitst worden. Dit gaat op twee manieren. 

Van `Genres` wordt een lijst gemaakt, alle genres zijn namelijk opgebroken met behulp van een `|`. Dit resulteerd in de kolom `Split Genres`. Hetzelfde proces wordt toegepast bij `Plot Keywords`. Dit resulteerd in de kolom `Split Keywords`.

Bij `Plot Keywords` worden alle `|` vervangen met een spatie. Dit wordt later verder toegelicht.

Hoewel `Split Keywords` niet wordt gebruikt als input variabele, is deze alsnog handig om te hebben bij het analyseren van de data. 

In [ ]:
# Split "Genres" by "|", creating list types
df_Q2.loc[:, "Split Genres"] = df_Q2["Genres"].str.split(pat="|")

# Replace "|" with " " to create strings analysable by TF-IDf.
df_Q2.loc[:, "Plot Keywords"] = df_Q2["Plot Keywords"].str.replace("|", " ")
# Split "Plot Keywords" by " ", creating list types
df_Q2.loc[:, "Split Keywords"] = df_Q2["Plot Keywords"].str.split(pat=" ")


df_Q2.head()

In [ ]:
pd.DataFrame([df_Q2["Split Genres"].str.len().describe(), df_Q2["Split Keywords"].str.len().describe()])

Er zijn gemiddeld drie genres en vijf plot keywords per film. Het is echter géén uniforme data. Zowel genres als plot keywords heeft een minimum van 1 woord (I.E. er is maar één genre of één plot keyword). Het maximaal aantal waarden voor genres is 8, het maximaal aantal waarden voor plot keywords is daarentegen 25. 


Een probleem met de vraagstelling is dat genres multilabel waardes zijn, een film dus heeft meer dan een genre. Alleen het eerste genre pakken is een mogelijkheid, maar dit zal de accuratesse van ons model zeer negatief beïnvloeden. De genres zijn namelijk niet gesorteerd op toepasbaarheid, maar alfabetisch. 

In [ ]:
alphabetical = True
for value in df_Q2["Split Genres"].values:
    if value != sorted(value):
        alphabetical = False
        
alphabetical
    

Zowel genres als plot keywords is categorische data. Aangezien meeste machine learning modellen algebraïsch zijn, accepteren deze alleen numerieke data. Dat betekent dat zowel genres als keywords omgezet moet worden naar numerieke data. 

Dit wordt meestal doormiddel van ‘One-Hot’ encoding gedaan. Bij 'One-Hot' encoding wordt van iedere unieke waarde een eigen kolom gemaakt. Deze kolom heeft een waarde '1' of '0'. Of de waarde komt voor in `Genres`, of niet.

Dit veroorzaakt echter een probleem wanneer er sprake is van een grote variatie aan herhalende data. In deze scenario kan term frequency-inverse document frequency (TF-IDF) gebruikt worden.

TF-IDF genereerd namelijk waardes en kolommen gebaseerd op de frequentie van woorden in een string.

De volgende stap is om te kijken naar unieke waarden van zowel `Genres` als `Plot Keywords`. Gebaseerd op het aantal unieke waarden wordt de de codeertechniek gekozen.

In [ ]:
genre_uniques = find_uniques(df_Q2["Split Genres"])
genre_uniques.describe()

Het gaat hierbij dus om 24 unieke genres. Ook zullen genres niet meer dan één keer in een rij voorkomen. Bij `Genres` zal dus One-Hot encoding gebruikt worden.

In [ ]:
key_uniques = find_uniques(df_Q2["Split Keywords"])
key_uniques.describe()

Er zijn in totaal 6104 unieke plot keywords. Bepaalde plot keywords zullen ook meerdere keren per rij voor kunnen komen. Bij `Plot Keywords` zal dus TF-IDF gebruikt worden.
#### Q2 Coderen van categoriale data
Beginnend met `Genres`, deze kolom wordt gecodeerd met behulp van One-Hot encoding.

In [ ]:
df_Q2 = one_hot(df_Q2, "Split Genres")
Y = df_Q2.loc[:, "Drama" : ]
Y.shape

Ten tweede coderen we `Plot Keywords`, deze kolom wordt gecodeerd met behulp van TF-IDF. 
TF-IDF kan ook veel gebruikte, maar weinig zeggende woorden (`stop words`) als ‘of’ en ‘and’ uit een tekst halen. Deze worden direct door de TF-IDF-vectorizer van Scikit-learn geëxtraheerd. Hierdoor neemt het aantal kolommen significant af.


In [ ]:
vec = TfidfVectorizer(stop_words=stop_words)
X = vec.fit_transform(df_Q2["Plot Keywords"])
X.shape


Hoewel 5994 kolommen nog steeds veel is, bespaar je als nog erg veel in vergelijking tot 6104 kolommen.

#### Q2. Processing van extra dataset

De data van genres staat al One-Hot gecodeerd. Er zijn hier echter wel meer genres dan bij de originele dataset. De extra genres moeten dus verwijderd worden.

In [ ]:
Y_genres = df_genres[np.intersect1d(Y.columns.values, df_genres.columns.values)].copy()
Y_genres.columns.size, Y.columns.size

Er mist nu nog één genre in `Y_genres`. Deze zal overal gelijk staan aan 0.

In [ ]:
missing_genre = np.setdiff1d(Y.columns.values, Y_genres.columns.values)
print("Missing genre:", missing_genre[0])
Y_genres.loc[:, missing_genre[0]] = 0
Y_genres = Y_genres[Y.columns.values]
Y_genres.head()

In [ ]:
Y_genres.columns == Y.columns

Vervolgens moet ook de X van `DF_genres` verwerkt worden. Bij de bron van de dataset moet er eerst gefilterd worden op taal. Hier is dat niet nodig, aangezien de TF-IDF vectorizer gefit is op een Engelse dataset.

In [ ]:
X_genres = vec.transform(df_genres["plot"])
X_genres.shape

### Q2. Data cleaning
Al het Data Cleaning is voor deze stap al gedaan, namelijk in de paragrafen `3. Data Cleaning` en `Q2. Cleaning voor processing`.

### Q2. Data Exploration & Analysis

Nu is een goed moment om te kijken naar de X en Y (noteer de hoofdletter bij Y, de output is namelijk ook een matrix). Beginnend met de X.

De X moet wel eerst omgevormd worden naar een NumPy array. X wordt nu namelijk nog opgeslagen in de vorm van een sparse array. Dit om ruimte in het geheugen te besparen.

In [ ]:
pd.DataFrame(X.toarray(), columns=vec.get_feature_names()).describe()

Hier valt al direct een probleem te ontdekken, de waardes gegenereerd door TF-IDF liggen redelijk laag. Dit betekent dat er niet veel van dezelfde woorden langskomen. Dit kán mogelijk de waarde van de voorspellingen negatief beïnvloeden. Hier wordt echter tot zekere hoogte rekening mee gehouden met behulp van het gebruik van `stop_words`.

Vervolgens de Y. Aangezien er een eigen One-Hot functie geschreven is, wordt Y niet opgeslagen in de vorm van een sparse array. Y is in de vorm van een DataFrame.


In [ ]:
Y.describe()

Aangezien alle waarden of `1` of `0` zijn, staat het gemiddelde ook gelijk aan het percentage films dit genre tot behoort. Zowel té hoog als té laag zal waarschijnlijk een slechte invloed hebben op de voorspellingen. 

Aangezien iets meer dan 50% van de films behoren tot het genre `Drama` behoren, zal er waarschijnlijk sprake zijn van veel false positives. 

Ook zijn er een groot aantal genres waar minder dan 5% van de films tot behoren. Aangezien hier relatief weinig data van is, zal het verbanden vinden tussen plot keywords en deze genres aanzienlijk moeilijker zijn. Waarschijnlijk zal er sprake zijn van veel false negatives. 


#### Extra data set
Tenslotte wordt er ook nog gekeken naar het aantal films per genre bij de extra dataset. 

In [ ]:
unique_extra = pd.DataFrame(Y_genres.sum(), columns=["Count"])
unique_extra.reset_index(inplace=True)
unique_extra.columns = ["Split Genres", "Test Count"]

unique_extra = unique_extra.merge(genre_uniques)
unique_extra.columns = ["Split Genres", "Test Count", "Train Count"]
unique_extra.sort_values("Test Count", ascending=False)

### Q2. Model building


Films zoals “[Bad Boys](https://www.imdb.com/title/tt0112442/?ref_=ttkw_kw_tt)” met genres als ‘Action, Comedy, Crime’ zullen dan alleen geclassificeerd worden als ‘Action’. [Plot keywords](https://www.imdb.com/title/tt0112442/keywords?ref_=tt_ql_stry_4) zoals ‘evil man’ en ‘firearm’ komen dan nog goed overeen met het genre, maar plot keywords als ‘buddy movie’ en ‘buddy comedy’ niet.

Om dit op te lossen, moet er gebruik gemaakt worden van ‘multi-label classification’. Dit is niet te verwarren met ‘multi-class classification’. Bij multi-label is er sprake van meerdere labels die tegelijkertijd toepasbaar kunnen zijn, bij multi-class is er altijd maar één klasse(label) toepasbaar.


Er worden voor deze onderzoeksvraag drie modellen toegelicht.

Ieder model wordt op drie manieren getest:
* Accuracy Score per genre (> == beter)
* Precision Score per genre (> == beter)
* Hamming loss op de gehele voorspelling (< == beter)

Tenslotte wordt de hoeveelheid films per genre als extra kolom toegevoegd.

Ten eerste, het opsplitsen in `test` en `train` datasets:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)
print("Train:", X_train.shape, Y_train.shape)
print("Test:", X_test.shape, Y_test.shape)

#### Q2. Imiteren met behulp van Multi-Class predictions
Eén methode om multi-label te implementeren, is om dit niet te doen. In plaats van “echte” multi-label classificatie te gebruiken, imiteer je het met behulp van Multi-Class classificatie. Voor iedere klasse train en test je het model apart. Dit komt echter met een forse performance hit. Ook zorgt dit ervoor dat voor iedere test set opnieuw het model moet laten fitten.

Qua model wordt er hier gebruik gemaakt van een Multinomial Naive Bayesian model, met daar omheen een veel gebruikte Multi-Class wrapper (OneVsRest). Multinomial Naive Bayesian is vrijwel de standaard voor text classificatie, en zal waarschijnlijk het eerste resultaat zijn indien je text classificatie googled.


In [ ]:
NB_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

predictions = []
for genre in genre_uniques["Split Genres"]:
    NB_pipeline.fit(X_train, Y_train[genre])
    predictions.append(NB_pipeline.predict(X_test))
   
predictions = np.array(predictions).T

In [ ]:
print("Hamming loss:", hamming_loss(Y_test, predictions))
column_score(Y_test, predictions).merge(genre_uniques, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)

De waarde van de voorspelling lijkt beter te worden naarmate het aantal films per genre het gemiddeld aantal films per genre (564, zie Q2. Data processing) bereikt. Zodra het aantal films hier ruim onder begint te komen, voorspelt dit model dat alle films tot die genres behoren.

De Hamming loss van dit model ligt erg laag, op ongeveer 0.11. Dat betekent dat de voorspeling van redelijk goede kwaliteit is.  


#### Q2. Binary Relevance en Naive Bayes classifiers
De tweede methode om multi-label classification toe te passen, heet Binary Relevance. Dit valt beter uit te leggen met behulp van een voorbeeld. Zie daarvoor de volgende paragraaf. Binary Relevance is op moment van schrijven nog niet opgenomen in het officiële Scikit-Learn library. Om dit te kunnen gebruiken is de Scikit-Learn addon ` skmultilearn` benodigd.
##### Multinomial
Hieronder wordt net zoals bij `Q2. Imiteren met behulp van Multi-Class predictions` gebruik gemaakt van Multinomiale Naive Bayesian classificatie. Met de `BinaryRelevance` wrapper van `skmultilearn` er omheen.

In [ ]:
MNB_classifier = BinaryRelevance(MultinomialNB())
MNB_classifier.fit(X_train, Y_train)

In [ ]:
predictions = MNB_classifier.predict(X_test)

In [ ]:
print("Hamming loss:", hamming_loss(Y_test, predictions))
df_MNB_score = column_score(Y_test, predictions).merge(genre_uniques, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)
df_MNB_score

De resultaten van Binary Relevance op Multinomial NB zijn exact hetzelfde als de resultaten resultaten van de eerder gebruikte pipeline. 

Binary Relevance valt binnen multi-label machine learning in het groepje `Problem transformation approaches`. Het pakt een multi-label probleem, en splitst deze op in kleinere problemen die op te lossen zijn met behulp van al bestaande technieken. 
Een nadeel hiervan is echter wel dat de verbanden tússen de verschillende labels buiten beschouwing raken. Het resultaat is desalniettemin beter dan verwacht.


##### Gaussian naive bayes

Een tweede model dat regelmatig wordt gebruikt voor het classificeren van tekst, is het Gaussian Naive Bayes model. Dit gaat uit van een normaalverdeling in plaats van een multinomiale verdeling.

Gaussian is meestal een aanzienlijk slechtere methode om tekst mee te classificeren, het wordt bij deze onderzoeksvraag alleen gebruikt om de kwaliteit van Multinomial Naive Bayes beter te laten zien.


In [ ]:
GNB_classifier = BinaryRelevance(GaussianNB())
GNB_classifier.fit(X_train, Y_train)

In [ ]:
predictions = GNB_classifier.predict(X_test)

In [ ]:
print("Hamming loss:", hamming_loss(Y_test, predictions))
df_GNB_score = column_score(Y_test, predictions).merge(genre_uniques, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)
df_GNB_score

Zoals verwacht is de voorspelling minder accuraat dan die van Multinomial NB. De voorspelling is echter wel beter dan verwacht. Het is beter in het voorspellen van de waardes die onder het gemiddelde vallen dan Multinomial NB, voor alle andere waarden lijkt het echter slechter te zijn. 

Tenslotte zijn de laatste drie genres (Film-Noir, Short & News) alle drie nog steeds niet mogelijk om te voorspellen.

#### Q2. Model testing; Extra dataset

Tenslotte wordt dezelfde voorspelling nog een keer uitgevoerd op de extra dataset. Hier wordt weer gebruik gemaakt van de Multinomial Naive Bayesian classifier uit `Q2. Multinomial`.


In [ ]:
predictions = MNB_classifier.predict(X_genres)

In [ ]:
print("Hamming loss:", hamming_loss(Y_genres, predictions))
df_extra_score = column_score(Y_genres, predictions).merge(unique_extra, left_on="Category", right_on="Split Genres").drop("Split Genres", axis=1)
df_extra_score

Hoewel de Hamming loss lager is dan bij de andere voorspellingen, is de precision score aanzienlijk slechter. De mindere waarde is op een paar manieren te verklaren.

De grootte van de originele dataset was te klein om op te trainen. Deze externe dataset is completer, maar daar wordt geen rekening mee gehouden bij het TF-IDF-proces. Er zullen waarschijnlijk veel termen voorkomen in de externe dataset, die niet voorkomen in de originele dataset.

De aard van de externe data is anders dan van het origineel. De externe data bestaat uit een uitgebreide beschrijving van het plot. De originele dataset bevat meerdere enkele termen of korte zinnen. Het kan zijn dat ‘keywords’ zodanig anders zijn dan een beschrijving van het plot, dat dat de voorspelling aanzienlijk negatief beïnvloed.

Het lijkt echter wel dat deze voorspelling hetzelfde pattern volgt als de voorspelling op de testdata. De precision wordt langzamerhand beter totdat deze het genre met het gemiddeld aantal films bereikt. Wanneer het onder dit gemiddelde valt, wordt de precision score 0.

### Q2. Visualization
Om een beter beeld te geven van de verbanden tussen het aantal films per genre, en hun accuracy- en precision scores, worden deze samen geplot.

In [ ]:
sc_MNB_ac = hv.Scatter(df_MNB_score[["Count", "Accuracy Score"]]).opts(title="MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_GNB_ac = hv.Scatter(df_GNB_score[["Count", "Accuracy Score"]]).opts(title="GNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_extra_ac = hv.Scatter(df_extra_score[["Train Count", "Accuracy Score"]]).opts(title="Extra MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)


sc_MNB_pr = hv.Scatter(df_MNB_score[["Count", "Precision Score"]]).opts(title="MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_GNB_pr = hv.Scatter(df_GNB_score[["Count", "Precision Score"]]).opts(title="GNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)
sc_extra_pr = hv.Scatter(df_extra_score[["Train Count", "Precision Score"]]).opts(title="Extra MNB", ylim=(-0.05, 1.05), xlim=(-100, 2500), size=5)


(sc_MNB_pr + sc_GNB_pr + sc_extra_pr + sc_MNB_ac + sc_GNB_ac + sc_extra_ac).cols(3)

### Q2. Communication
#### Q2. Definitieve pipeline
De definitieve pipeline van dit project is de `MNB_classifier`. De Binary Relevance Multinomial Naive Bayesian classifier. Van de twee classificatie modellen, geeft deze betere voorspellingen.
#### Q2. Conclusie
De onderzoeksvraag die beantwoord wordt met behulp van dit model ging als volgt:
> In hoeverre is het mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort?

Het is zeer goed mogelijk om op basis van plot keywords te voorspellen tot welke genres een film behoort. Er zijn echter wel een paar dingen waar, bij dit model, rekening mee gehouden moet worden. 
Ten eerste is het model, met de huidige training data, niet goed in het voorspellen van genres met minder dan 515 datapunten. Dat houdt dat maar 10 van de 24 genres correct voorspeld kunnen worden. 

De meeste films zullen in principe behoren tot die 10 genres, maar dat betekent niet dat er geen films van die genres bestaan. Bij een dataset van circa 5000 films is dit nog geen groot probleem, bij volledigere datasets (denk aan 100K+) zal dit echter wél voor een probleem veroorzaken.

Ten tweede houdt dit model geen rekening met de onderlinge verbanden tussen genres. Een film behorend tot het genre ‘Biography’ zal vaak ook behoren tot het genre ‘Documentary’. 

#### Q2. Mogelijk vervolg onderzoeken

Bij een mogelijk vervolgonderzoek komen een aantal dingen aan bod. Ten eerste zou dit onderzoek opnieuw uitgevoerd moeten worden op een andere, grotere dataset. 

Ten tweede zou er gekeken moeten worden naar meer geavanceerde multi-label classificatie methoden die wél rekening houden met de onderlinge verbanden tussen genres.

Ten derde zou er gekeken moeten worden naar de verschillen tussen een plot beschrijving (zoals van de externe dataset) en plot keywords (van de originele dataset).


#### Q2 Extra leesmateriaal
[TF-IDF](http://www.tfidf.com)

[Understanding Multi-Label classification model and accuracy metrics](https://medium.com/towards-artificial-intelligence/understanding-multi-label-classification-model-and-accuracy-metrics-1b2a8e2648ca)

[Multi Label Text Classification with Scikit-Learn](https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5)

[Binary relevance for multi-label learning: an overview](https://link.springer.com/article/10.1007/s11704-017-7031-7)

***

## Q3. Budget-omzet verhouding voorspellen
Ook voor deze onderzoeksvraag zullen de zeven stappen van het Data Science-proces worden toegepast. 
Stappen 1, 2 en 3 zullen in grote maten hetzelfde zijn als bij de overige twee onderzoeksvragen.
### Het Data Science-proces:
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
7. Communication

***

### Q3. 1. Data Collection
Voor het beantwoorden van deze onderzoeksvraag zal alleen gebruik worden gemaakt van `movie.csv`.
Om te zien of de dataset `movie.csv` goed is ingeladen, worden de eerste vijf rijen getoond:

In [ ]:
df_Q3 = df_movies.copy()
print(f"In totaal zijn er nog {len(df_Q3)} films over na de algemene Data Processing en Data Cleaning stappen.\n")
df_Q3.head()

Te zien is dat de data goed wordt ingelezen en de waardes in de juiste kolommen komen te staan. Verder zijn er geen NaN-waardes te zien en staan veel gegevens niet meer opgeslagen als floats. Tijdens de algemene _Data Cleaning_ is al een hoop voor ons gedaan. Echter moet de data voor deze specifieke onderzoeksvraag nog verder worden aangepast en opgeschoond tijdens de _Data Processing_ en _Data Cleaning_ stappen.

***

### Q3.  2. Data Processing
Ook de Data Processing is grotendeels voor ons gedaan. De data is goed opgeslagen in een `.csv`-bestand
en is zojuist al ingelezen als een _Pandas_ DataFrame.

Verder rest ons nog de volgende stappen om de Data Processing te voltooien:
1. Het selecteren van de gewenste kolommen

2. Het berekenen van de budget-gross verhouding van iedere film
3. Het berekenen van de Z-score-afstand tussen budget en film 
4. Het berekenen van de Z-scores van de feature-kolommen

Alvorens de gewenste kolommen kunnen worden geselecteerd, moet worden gekeken welke kolommen überhaupt relevant zijn. Pandas heeft een hele gemakkelijke functie om voor een compleet DataFrame alle correlaties uit te rekenen. De correlaties worden standaard volgens Pearsons correlatiecoëfficient berekend. Vervolgens kan het correlatiematrix gemakkelijk worden geplot met Seaborn. Seaborn maakt het mogelijk om een snel overzicht te krijgen van de correlaties.

> NB. Uiteraard is het pas bij de _Data Exploration_ van belang om de data volledig te doorgronden, echter moet tijdens de _Data Processing_ al worden gekozen welke kolommen gebruikt gaan worden. Aangezien Data Science een iteratief proces is kan tijdens een later stadium ervoor worden gekozen om meer kolommen te behouden.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(df_Q3.corr(), annot=True, linewidths=.05, ax=ax, square=True)
plt.show()

Na het bestuderen van het geplotte correlatiematrix blijkt dat `Number of voted users`, `Number of user reviews` en `Number of critics` de grootste correlatie hebben met `Budget` en `Gross`. De twee laatstgenoemden zijn uiteraard vereisten voor het beantwoorden van de onderzoeksvraag. De verhouding tussen het budget en de omzet moet immers worden voorspeld.

Verder selecteren we nog de kolom `Release year`. Dit zou eventueel later van pas kunnen komen voor het sorteren of visualiseren van de data. Zoals eerder genoemd zou het mogelijk kunnen zijn dat in een volgende iteratie blijkt dat er meer kolommen nodig zijn.

In [ ]:
# Selecteren gewenste kolommen
df_Q3 = df_Q3[["Movie title", "Budget", "Gross",
               "Release year", "Number of voted users",
               "Number of user reviews", "Number of critics"]]

print(f"Kolommen:")
df_Q3.columns.tolist()

Integers zijn een stuk sneller dan floats. Het komt de efficiëntie en netheid ten goede om zo veel mogelijk kolommen naar integers
om te zetten.

NaN-types kunnen echter niet worden opgeslagen in kolommen met het datatype `int32`. Als oplossing is ervoor gekozen om alle NaN-types om te zetten naar integers. De meest logische keuze zal 0 zijn.

Het datatype `Int64` geeft wel de mogelijkheid om NaN-types samen met integers op te slaan. Echter is het `Int64`-datatype nog experimenteel en is de afweging gemaakt om dit datatype niet te gebruiken.

[Bron: Nullable integer data type](https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html)


In [ ]:
# Datatypen aanpassen: Floats naar integers omzetten
df_Q3.fillna(0, inplace=True)
df_Q3 = df_Q3.astype({"Budget": int,
                      "Gross": int,
                      "Number of voted users": int, 
                      "Number of user reviews": int,
                      "Number of critics": int})

df_Q3.dtypes

Het beantwoorden van de onderzoeksvraag vraagt naar het voorspellen van de budget/omzet-verhouding.
De kolom `Budget Gross verh` bevat de mate waarin `budget` en `gross` zich verhouden. Dit werkt middels de simpele formule $\frac {Gross} {Budget}$. Een waarde groter dan `1` betekend winst. Er wordt dan meer omzet gedraaid dan dat er budget was de film. Alles onder de `1` betekend verlies. Hierbij is de omzet lager dan het budget.

`Budget Gross Zscore afst` houdt het (absolute) verschil tussen de Z-score van het budget
en de gross in. Een klein voorbeeld ter illustratie:
> - Het budget van een film heeft een Z-score van 1
> - De omzet van dezelfde film heeft een Z-score van 4
>
> De `Budget Gross Zscore afst` van deze film bedraagt $(|4-1|)=$ $3$  
> Omdat de afstand minder dan 6 is kan worden geconcludeerd dat het onderlinge verschil niet significant en de film mag blijven in de dataset.
  
Het verwijderen van waardes met een absolute Z-score afstand van meer dan 6 zal later gebeuren tijdens de _Data Cleaning_. Door deze significante waardes te verwijderen uit de dataset zullen betere voorspellingen mogelijk worden. Uitschieters kunnen de precisie van een model ernstig verslechteren.

In [ ]:
# Verhouding tussen budget en gross berekenen en opslaan in nieuwe kolom
df_Q3["Budget Gross verh"] = df_Q3["Gross"] / df_Q3["Budget"]

# Zscores berekenen van Budget en Gross.
df_Q3["Budget Gross Zscore afst"] = abs(zscore(df_Q3[["Budget"]]) - zscore(df_Q3[["Gross"]]))

# Z-scores berekenen van de mogelijke features
df_Q3["Number of voted users Zscore"] = abs(zscore(df_Q3[["Number of voted users"]]))
df_Q3["Number of user reviews Zscore"] = abs(zscore(df_Q3[["Number of user reviews"]]))
df_Q3["Number of critics Zscore"] = abs(zscore(df_Q3[["Number of critics"]]))

df_Q3[["Budget Gross verh", "Budget Gross Zscore afst",
       "Number of voted users Zscore",
       "Number of user reviews Zscore",
       "Number of critics Zscore"]].describe()

Het DataFrame `df_Q3` ziet er nu als volgt uit:

In [ ]:
df_Q3.head()

Alle kolommen van het DataFrame hebben nu het juiste datatype.
Onder andere `Budget` en `Gross` bestaan uit integers en `Release year` bestaat uit het `datetime` datatype.
Echter zijn er nog wel 0-, NaN, en NaT-waardes zichtbaar. Deze rijen dienen tijdens de _Data
Cleaning_ nog te worden opgeschoond.


***

### Q3.  3. Data Cleaning
Nu de data in de juiste kolommen met de juiste datatypes is opgeslagen kunnen we de data gaan opschonen.
Na het uitvoeren van de volgende vijf stappen hebben we een DataFrame dat klaar voor analyse is.
1. Het omzetten van onrealistische waardes naar 0
2. Het verwijderen van rijen met een `Budget Gross zscore verh` van meer dan 6
3. Het verwijderen van films met een budget of gross van 0
4. Het verwijderen van NaN-types


In [ ]:
print(f"In totaal zijn er {len(df_Q3)} films vóór de Data Cleaning.")

# Onrealistische waardes omzetten
print(f"In totaal zijn er {df_Q3[['Budget']].gt(400_000_000).sum().sum()} films met een budget hoger dan de duurste film ooit.")
print(f"In totaal zijn er {df_Q3[['Budget', 'Gross']].gt(2_800_000_000).sum().sum()} films met een hogere omzet dan de meest succesvolle film ooit.")
print(f"In totaal zijn er {df_Q3['Budget Gross Zscore afst'].gt(6).sum().sum()} films met een extreem verschil tussen budget en gross.")

# 'Te dure' films verwijderen
df_Q3 = df_Q3[(df_Q3["Budget"] < 400_000_000)]

# Verwijderen films met Z-score afstand van meer dan 6
df_Q3 = df_Q3[(df_Q3["Budget Gross Zscore afst"] <= 6)]

# Verwijderen films met budget of gross van 0
df_Q3 = df_Q3[(df_Q3[["Budget", "Gross"]] != 0).all(axis=1)]

# NaN-types verwijderen uit de titels
df_Q3.dropna(inplace=True)
      

print(f"In totaal blijven er {len(df_Q3)} films over.")

Om te controleren of het DataFrame volledig is opgeschoond wordt `.describe()` aangeroepen om een beschrijving te krijgen van iedere kolom.

Twee zaken lijken ernstig onrealistisch en dienen te genieten van nader onderzoek:
- De kolom `Budget Gross verh` bevat een film die meer dan 7000x opbracht dan het budget
- De drie Z-score featurekolommen bevatten extreme outliers met Z-scores van wel meer dan 10

In [ ]:
df_Q3.describe(include="all")

Om erachter te komen waar deze outliers vandaan komen worden de individuele films bekeken. Alle films met een verschil van meer dan 500x worden in onderstaande tabel getoond.

Tegen de verwachtingen in blijken het `Budget` en `Gross` te kloppen. Deze films zijn daadwerkelijk realistische outliers en dienen te worden bewaard in het DataFrame. Alle drie de films hebben een eigen verhaal waarom het budget extreem laag is. Vooral The Blair Witch Project en Paranormal Activity zijn gigantisch doorgebroken en beroemde films geworden.

In [ ]:
df_Q3[(df_Q3["Budget Gross verh"] > 500)]

Ook de extreme Z-scores van onderstaande films blijken te kunnen worden verklaard en zijn correct. The Shawshank Redemption staat al jaren op nummer 1 in de IMDB-database. Hierdoor geniet de film nog steeds van veel aandacht. Nog steeds worden er veel user- of critic-reviews geschreven voor de film. De andere twee films staan tevens hoog in de ranglijst van IMDB. Ook voor deze twee vallen de extreem hoge Z-score te verklaren.

In [ ]:
df_Q3[(df_Q3[["Number of voted users Zscore",
              "Number of user reviews Zscore",
              "Number of critics Zscore"]] > 10).any(axis=1)]

Na het valideren van het eindresultaat zit _stap 3. Data Cleaning_ er op. Het DataFrame ziet er nu als volgt uit:

In [ ]:
df_Q3.head()

Het DataFrame is nu volledig opgeschoond en klaar voor de _Data Exploration & Analysis_.

Het DataFrame bevat alleen maar nog unieke waardes voor `Movie title`.
Door het DataFrame te sorteren op `Release year` en vervolgens de laatste waarde te behouden, wordt
de meest recente uitgave van een film bewaard. Te zien is dat maar liefst 126 films dubbel
in de dataset voorkwamen.

Het omzetten van alle onrealistische waardes van `Budget` en `Gross` worden vervolgens omgezet naar `0`. Onrealistische waardes van `Budget` en `Gross` zijn onder andere negatieve waardes. Andere onrealistische waardes zijn films met een `Budget` en/of `Gross` van meer dan 2.8 miljard. Er is immers nog nooit een film* geweest die meer dan 2.8 miljard opbracht.
Verder werden alle films met een `Budget` van meer dan 400 miljoen gedropped, de duurste film ooit
kostte niet meer dan 400 miljoen. Door deze stap worden enkele extreme outliers gefilterd uit
de data wat de voorspellingen en visualisatie ten goede zal komen. 

[Bron: Lijst van succesvolste films](https://nl.wikipedia.org/wiki/Lijst_van_succesvolste_films )

Uiteindelijk worden alle films met een `Budget` of `Gross` van 0 verwijderd uit het DataFrame.

Zoals beschreven bij de _Data Processing_ zijn alle waardes met een significant verschil tussen de Z-scores van `Budget` en `Gross` verwijderd. Zie voor meer toelichting de documentatie bij _Data Processing_.

***

### Q3.  4. Data Exploration & Analysis

Na het opschonen kan begonnen worden aan de eerste verkenning. Door middel van describe krijgen we in één oogopslag een duidelijk beeld van het DataFrame:

In [ ]:
df_Q3.describe(include="all")

Enkele verwachtingen:
1. Het meerendeel van de films zal winst maken
2. Nieuwere films zullen relatief meer winst maken dan oudere films
3. Het budget van relatief oudere films zal lager zijn dan dat van relatief nieuwe films 

Enkele eerste observaties:
- Het DataFrame bevat nog 3772 (74.8%) van de originele 5043 rijen 
- `Movie title` bevat zoals beoogd alleen maar nog unieke waardes.
- Gemiddeld brengt een film 50 miljoen op en is er een budget van 37 miljoen. Echter heeft
 het budget een standaard deviatie van 42 miljoen en de omzet een standaard deviatie van 66 miljoen.
- Het `Release year` 2002 komt met 189x het vaakst voor
- Het DataFrame bevat films die uitgegeven zijn tussen 1920 en 2016. Een interval van bijna 100 jaar.

Het opstellen van verwachtingen en het doen van observaties kan men op ideeen brengen. Tijdens de _Data Exploration_ worden vaak de eerste interpretaties van de data gemaakt. Verwachtingen en observaties kunnen leiden tot nieuwe invalshoeken voor het bestuderen van de onderzoeksvraag.

***

In de volgende scatterplot zijn het budget en de omzet van films geplot.
Dit geeft een eerste beeld van de spreiding van de data en de relatie tussen het budget en de omzet.
Er wordt een sample van 350 (+- 10%) stuks genomen om de data beter te visualiseren en overzichtelijk te houden.

Tevens wordt de baseline getekend. Deze wordt vastgesteld op $y = x$ oftewel het break-even point.
Alles films rechts van de lijn hebben verlies geleden, allen links van de lijn maken winst. De baseline
wordt gebruikt om vast te stellen of een geproduceerd model wel echt beter presteert dan geen model of een snelle gok.

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
df_Q3_sample = df_Q3.sample(350, random_state=42)
line = np.linspace(0, 400_000_000)

ax.plot(line, line, zorder=0, color='black', label="Baseline")
sns.scatterplot("Budget", "Gross", data=df_Q3_sample, label="Films", ax=ax)

plt.grid(True)
plt.xlim(0, 400_000_000)
plt.ylim(0, 400_000_000)
plt.xlabel("Budget (x100 mln)")
plt.ylabel("Gross (x100 mln)")
plt.legend(loc="lower right")
plt.title("Scatterplot tussen Budget en Gross"
          "\nBaseline: Y = X")
plt.show()

Een mogelijke functie om, in plaats van een selectie, alle films te analyseren is `jointplot`. Jointplots geven op een snelle manier de spreiding van punten weer. Er is goed te zien dat de meeste films onder een budget van één miljoen blijven en dat veruit de meeste films onder een omzet van twee miljoen blijven. Er valt zelfs al een kleine positieve correlatie te spotten.

Om de plot overzichtelijk te houden is gekozen om de dichtheid weer te geven. Het grote aantal films kan, zoals we hierboven al zagen, tot een onoverzichtelijke plot leiden.
[Bron: Seaborn Jointplot](https://seaborn.pydata.org/generated/seaborn.jointplot.html)

In [ ]:
sns.jointplot("Budget", "Gross", data=df_Q3, kind="kde", space=0)
plt.show()

In de vorige jointplot waren individuele outliers niet te zien. Boxplots zijn uitermate geschikt om deze wel weer te geven.

De omzet van films is veel verspreider dan het budget. Tevens zijn er bij de omzet veel meer extreme outliers te zien. Door het grote aantal uitschieters kunnen zelfs boxplots heel onoverzichtelijk worden. Te zien is dat het derde kwartiel van beide kolommen onder de 100 miljoen liggen. Het maximum, zijnde Q3 + 1.5 * IQR, bevindt zich op ongeveer 100 resp. 150 miljoen voor de kolommen.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
df_Q3[["Budget", "Gross"]].boxplot(grid=False, ax=ax)
plt.ylim(bottom=0)
ax.yaxis.grid()

plt.ylabel("x100 mln")
plt.title("Boxplots van Budget en Gross")
plt.show()

Een laatste poging om ook de outliers in kaart te brengen wordt gedaan door twee
histogrammen te plotten. Bovenop de histogrammen worden streepjes getekend. Ieder streepje is een
film. De outliers zijn nu zichtbaar geworden. Direct valt te zien dat het budget
twee extreme uitschieters heeft bij 300 en resp. 400 miljoen.
Gross heeft uitschieters die door gaan tot zo'n ongeveer 550 miljoen.

Ook hier wordt weer duidelijk dat het merendeel van de films het budget onder de 100 miljoen houdt
en de opbrengst vooral onder de 200 miljoen blijft. Er zijn slechts enkele films met een extreme
verhouding tussen het budget en de omzet. `Number of critics` concentreert zich vooral van 0 tot 400.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 3), sharex="row", sharey="row")
sns.distplot(df_Q3["Budget"], rug=True, kde=False, ax=axes[0], color="r")
sns.distplot(df_Q3["Gross"], rug=True, kde=False, ax=axes[1], color="g")
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(12, 3))
sns.distplot(df_Q3["Budget Gross verh"], rug=True, kde=False, ax=axes[0], color="b")
sns.distplot(df_Q3["Number of critics"], rug=True, kde=False, ax=axes[1], color="cyan")
plt.show()

Pandas heeft een hele handige methode ingebouwd voor DataFrames. Deze methode berekend direct de
correlatie tussen iedere combinatie van twee variabelen. Standaard wordt de correlatie
door middel van Pearsons correlatiecoëfficiënt berekend.

Volgens Pearsons correlatiecoëfficiënt blijkt dat er een correlatie van 0.64 is tussen Budget en
Gross. Verder geven `Number of voted users`, `Number of user reviews` en `Number of critics` ook redelijke correlaties bij
`Budget` en  `Gross`. Echter willen we voor het beantwoorden van de onderzoeksvraag de correlatie tussen
de verschillende features en `Budget Gross verh` weten.

> NB. Met het bekijken van de onderlinge correlaties is een goed beeld ontstaan van de te gebruiken features. Aangezien Data Science een iteratief proces is, zijn deze features tijdens de _Data Processing_ al geselecteerd. In een later stadium kan er nog worden gekozen om meer kolommen te selecteren als features.

In [ ]:
df_Q3.corr()

***

### Q3.  5. Model Building
Om een beeld te krijgen van de verhouding tussen het budget en de omzet bouwen we een Linear Regressie-model. Lineaire  regressie is vaak een mooi startpunt. Voor de X-as wordt het budget geselecteerd en voor de y-as de omzet. Vervolgens wordt de data gesplitst in een test- en trainset. Aangezien er redelijk veel data tot de beschikking is, wordt er getraind met een 80% / 20% verhouding.

In [ ]:
X = df_Q3[["Budget"]]
y = df_Q3["Gross"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
print(f"Aantal trainwaarden: {len(X_train):5}")
print(f"Aantal testwaarden: {len(y_test):6}")

Vervolgens dient het model te worden aangemaakt en getraind. Zodra dit model getraind is kan de formule $Y = AX + B$ worden ingevuld. Deze formule maakt het mogelijk om ieder mogelijk budget $x$ in te vullen waarna $y$ zal resulteren in de voorspelde omzet.

In [ ]:
linReg = LinearRegression()
linReg.fit(X_train, y_train)
print(f"Formule van de lijn:\nY = {linReg.coef_[0]:.4f} * X + {linReg.intercept_:,.4f}")

Zodra het model is gemaakt en getraind dient het te worden gevalideerd. Belangrijke prestatieindicatoren voor Lineaire Regressie zijn de $R^2$ en de Root Mean Squared Error (RMSE).
 

In [ ]:
y_pred = linReg.predict(X_test).reshape(-1, 1)
print(f"R2 score: {linReg.score(y_pred, y_test):.4f}")
print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred)):,.4f}")  # Werkt

Om bovenstaand model weer te geven wordt bovenop een scatterplot de vastgestelde lineaire formule getekend. Deze lijn wordt in het rood weergegeven. In het zwart is wederom de baseline te zien.

Te zien is dat de vastgestelde formule, de y-as bij een x-waarde van 0, hoger snijdt dan de baseline. Aangezien $A$ groter dan 1 is en B groter dan 0, zal de lijn de baseline nooit snijden. Uit de formule valt te concluderen dat hoe groter het budget, hoe relatief meer de winst zal worden. In het loop van de tijd neemt de budget/omzet-verhouding toe. 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

line = np.linspace(0, 400_000_000)
ax.plot(line, line, zorder=1, color='black', label="Baseline")

X_line = np.linspace(0, 400_000_000)
y_line = linReg.coef_ * X_line + linReg.intercept_  # y = ax+b
ax.plot(X_line, y_line, label=f"Correlatie: {linReg.score(X_test, y_test):.4f}", zorder=1, color="red")

ax = sns.scatterplot(X_test["Budget"], y_test, label="Films")

plt.grid(True)
plt.xlim(0, 400_000_000)
plt.ylim(0, 400_000_000)
plt.xlabel("Feature: Budget (x100 mln)")
plt.ylabel("Target: Gross (x100 mln)")
plt.legend(loc="lower right")
plt.title("Scatterplot tussen Budget en Gross met Lineaire Regressie\n"
          "Baseline: Y = X\n"
          "Formule van de lijn: \n"
          f"Y = {linReg.coef_[0]:.4f} * X + {linReg.intercept_:,.4f}\n")
plt.show()

### Tussentijdse evaluatie
Na het evalueren van bovenstaand model en het opnieuw bestuderen van de onderzoeksvraag zijn
we tot de conclusie gekomen dat bovenstaand model iets anders doet dan waar de onderzoeksvraag
naar vraagt. Met bovenstaand model kan, tot op zekere hoogte, de omzet worden voorspeld bij
een bepaald budget.

Waar de onderzoeksvraag echter naar vraagt is het voorspellen van de budget/omzet-verhouding.
Aan ons de taak om een goede combinatie van features te selecteren waardoor de 
budget/omzet-verhouding kan worden voorspeld. Tijdens de dataprocessing is een correlatiematrix
getoond waaruit bleek dat `Number of voted users`, `Number of user reviews` en `Number of critics`
de beste correlatie vertonen met `Budget` en/of `Gross`.

Er wordt een nieuw model opgesteld met deze set features als X. Verder wordt `Budget Gross verh`
als y genomen.

In [ ]:
X = df_Q3[["Number of voted users",
               "Number of user reviews",
               "Number of critics"]]
y = df_Q3["Budget Gross verh"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
print(f"Aantal trainwaarden: {len(X_train):5}")
print(f"Aantal testwaarden: {len(y_test):6}")

Ook zoals bij het vorige lineaire regressiemodel wordt het getraind en zal de formule van de lijn worden berekend:

In [ ]:
linReg = LinearRegression()
linReg.fit(X_train, y_train)
print(f"Formule van de lijn:\nY = {linReg.coef_[0]:.4f} * X + {linReg.intercept_:.4f}")

Ten slotte worden de R2 Score en de RMSE ook voor dit model berekend. Een R2 score van -4.4 is extreem laag maar wel mogelijk. Het is met aan zekerheid grenzende waarschijnlijkheid te stellen dat het huidige model met de huidige features geen correlatie heeft met `Budget Gross verh`. De drie features hadden individueel tijdens de Data Exploration echter wel enige maat van correlatie met `budget` en/of `omzet`.

[Bron: R2 value less than -1](https://stackoverflow.com/questions/23036866/scikit-learn-is-returning-coefficient-of-determination-r2-values-less-than-1)

In [ ]:
y_pred = linReg.predict(X_test)
print(f"R2 score: {linReg.score(X_test, y_test):.4f}")
print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred)):_.4f}")

Zojuist is gebleken dat de drie features samen geen correlatie vertonen met `Budget Gross verh` ondanks dat de individuele features correlatie vertoonde met `Budget` en/of `Gross`. De volgende code gaat voor alle mogelijke subsets van de drie features hun correlatie en RMSE berekenen. Door dit proces kan de beste combinatie van de drie features worden geselecteerd.

In [ ]:
X_combinaties = [["Number of voted users", "Number of user reviews", "Number of critics"],
                 
                 ["Number of voted users", "Number of user reviews"],
                 ["Number of voted users", "Number of critics"],
                 ["Number of user reviews","Number of critics"],
                 
                 ["Number of voted users"],
                 ["Number of user reviews"], 
                 ["Number of critics"]]

for features in X_combinaties:
    X = df_Q3[features]
    y = df_Q3["Budget Gross verh"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
    linReg.fit(X_train, y_train)
    y_pred = linReg.predict(X_test)
    
    print(f"Conbinatie: {X.columns.values}")
    print(f"R2 score: {linReg.score(X_test, y_test):23.4f}")
    print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}\n")

Te zien is dat de de feature `Number of voted users` met een R2 van -0.4520 het beste presteert. De waarde komt echter nog niet in de buurt van een perfect verband (R2 = 1). Het model is nog steeds onbruikbaar voor enige vorm van voorspellingen.

***

### Q3.  6. Visualization
Ondanks dat tijdens de _Model Building_ gebleken is dat de resultaten met betrekking tot de R2-score zeer slecht zijn, kunnen de resultaten alsnog worden geplot. In onderstaande visualisatie zijn meerdere trendlijnen zichtbaar. Voor iedere feature wordt een trendlijn getekend. Verder is de baseline $y=x$ nog zichtbaar ter vergelijking.

De coëfficiënten van `Number of user reviews Zscore` en  `Number of critics Zscore` zijn groter dan 1. Hierdoor zal de Budget/Omzet-verhouding steeds meer toenemen naarmate één van de features toeneemt. Voor `Number of voted users Zscore` zal de verhouding en dus de winst steeds meer afnemen totdat deze de baseline kruist. Deze feature heeft namelijk een coëfficiënten lager dan 1. Zodra de baseline is gepasseerd zal er verlies worden gemaakt.

In [ ]:
X_combinaties = [["Number of voted users Zscore"],
                 ["Number of user reviews Zscore"], 
                 ["Number of critics Zscore"]]

fig, ax = plt.subplots(figsize=(12,8))
line = np.linspace(0, 6, 755)
ax.plot(line, line, color='black', label=f"Baseline:")

for feature in X_combinaties:
    X = df_Q3[feature]
    y = df_Q3["Budget Gross verh"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=True, train_size=0.8)
    linReg.fit(X_train, y_train)
    
    X_line = np.linspace(0, 6)
    y_line = linReg.coef_ * X_line + linReg.intercept_  # y = ax+b
    
    ax.plot(X_line, y_line, label=f"{feature[0][:-7]} Correlatie: {linReg.score(X_test, y_test):.4f}")

plt.grid(True)
plt.xlim(0, 6)
plt.ylim(0, 80)
plt.xlabel("Z-score van de individuele features")
plt.ylabel("Target: Budget/Gross-verhouding")
plt.title("Lineair verband tussen de drie individuele features en de Budget/Gross-verhouding")
plt.legend()
plt.show()

Waarbij in bovenstaande plot de lineaire regressielijnen van de losse features worden geplot, zijn in onderstaande plot de lineaire regressielijnen van iedere combinatie van twee features zichtbaar. Aangezien we nu met twee features en één target-waarde bezig zijn, stappen we over naar 3D-plots.

In [ ]:
X_combinaties = [["Number of voted users Zscore", "Number of user reviews Zscore"],
                 ["Number of voted users Zscore", "Number of critics Zscore"],
                 ["Number of user reviews Zscore","Number of critics Zscore"]]

fig = plt.figure(figsize=plt.figaspect(0.333))
for i, features in enumerate(X_combinaties):
    X = df_Q3[features]
    y = df_Q3["Budget Gross verh"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
    linReg.fit(X_train, y_train)
    
    y_pred = linReg.predict(X_test)

    x_line = np.linspace(0, 10)
    y_line = linReg.coef_[0] * x_line + linReg.intercept_
    z_line = linReg.coef_[1] * x_line + linReg.intercept_
    
    ax = fig.add_subplot(1, 3, i+1, projection='3d')
    
    ax.scatter3D(X_test[features[0]], X_test[features[1]], y_pred)
    ax.plot3D(x_line, y_line, z_line, color="red")
    ax.set_title(f"{features[0]}\n{features[1]}")
    ax.set_xlabel(f"{features[0]}")
    ax.set_ylabel(f"{features[1]}")
    ax.set_zlabel("Budget/Gross-verhouding")

plt.tight_layout()
plt.show()

Helaas is het niet mogelijk om de lineaire regressielijn van drie features samen met één target weer te geven in een 4D-plot. Plotten in een 4-dimensionale ruimte is niet mogelijk. Wel kan er gekozen worden om in een 3D-plot als vierde marker te kiezen voor bijvoorbeeld de grootte, vorm of kleur van de individuele films.

Naast bovenstaande grafieken is de lineaire regressie van _Model building_ tussen het budget en de gross de meest interessante  visualisatie. In deze visualisatie zijn de individuele punten en tevens de eigenschappen van de lineaire lijn goed terug te zien. De jointplot die gemaakt is tijdens de _Data Exploration & Analysis_ geeft op een vereenvoudigde maar visueel aantrekkelijke manier de dichtheid weer.

***

### Q3.  7. Communication

Als pipelijn is deze onderzoeksvraag zijn de _7 stappen van het Data Scienceproces_ strikt gevolgd. Tijdens sommige stappen als de _Model Building_ komt de realisatie dat het project anders moet worden aangepakt. Wat volgt is het werpen van een nieuwe blik op de onderzoeksvraag en soms weer beginnen bij het whiteboard. Bij bijvoorbeeld de _Data Exploration_ bleek dat er meer kolommen nodig waren voor meer features. Aangezien Data Science een heel iteratief proces is, zijn deze kolommen bij de _Data Processing_ toegevoegd en vervolgens schoongemaakt tijdens de _Data Cleaning_.

Het documenteren en coderen van een interactief proces kan heel verwarrend worden. Om het proces duidelijk te houden is er voor deze onderzoeksvraag voor gekozen om de structuur en volgorde van de zeven stappen in stand te houden. Zoals net beschreven worden in een eerdere processtap soms al keuzes gemaakt die pas in een volgend stadium duidelijk worden.

***

Het continu valideren van de te gebruiken data is een heel belangrijk onderdeel om de kwaliteit van de te trekken conclusies te kunnen garanderen. Na de _Data Cleaning_ moet worden gekeken of de juiste data is verwijderd of juist is behouden. Verder dienen uitschieters te worden geanalyseerd en gevalideerd en moet de correcte data worden geselecteerd om te visualiseren. Niet zomaar alle data kan met elkaar worden getekend in een grafiek. Dit alles zal leiden tot een verhoogde betrouwbaarheid van het eindproduct.

Het selecteren en opschonen voor _Exploration_ en _Model building_ maken van de data is de meest iteratieve stap geweest voor het beantwoorden van de derde onderzoeksvraag. Continu dient de data te worden herzien en aangepast. Continu dient er data te worden verwijderd of toegevoegd. Continu dient er te worden gekeken of de data wel relevant is en er niet onjuiste conclusies worden getrokken.

***
### Beantwoording onderzoeksvraag, conclusie en aanbeveling

De derde onderzoeksvraag ging als volgt:
> #### In hoeverre is het mogelijk om de budget/omzet verhouding te voorspellen?

Het is volgens de huidige werkwijzen, met de huidige modellen, data en features, ons niet mogelijk geweest om de budget/omzet-verhouding zodanig te voorspellen dat hier betrouwbare waardes uit kunnen komen. Tijdens het onderzoek is de R2-waarde niet boven de 0,5% gekomen. De prestaties van alle modellen zijn zeer onder de maat en niet bruikbaar voor de praktijk.

Een mogelijke oorzaak kan zijn dat er nog steeds een aantal extreme outliers in de data zitten. Deze outliers zijn bekend maar zijn bewaard gebleven omdat de waardes allemaal kloppen. De outliers waren niet foutief. Verder was het opmerkelijk dat het budget en de omzet afzonderlijk wel tot op zekere hoogte zijn te voorspellen. Wanneer de verhouding tussen het budget en de omzet is berekend, valt de geresulteerde kolom ineens niet meer te voorspellen. Een mogelijke oplossing zou zijn om het budget en de omzet individueel te voorspellen in twee aparte modellen.

Mogelijk geïnteresseerden uit de business hebben mogelijk meer belang bij het voorspellen van de omzet (gross) ten opzichte van hun investering (budget). Mede hierom zou het in de toekomst interessanter zijn om de omzet te voorspellen in plaats van de budget/omzet-verhouding.